<a href="https://colab.research.google.com/github/ROBEASY/bdt-2024-17906342/blob/main/17906342_Tut3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
nyc_df = pd.read_csv("https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv")

<ipython-input-5-9211562249f3>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nyc_df = pd.read_csv("https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv")


In [6]:
print(len(nyc_df))

2118226


In [8]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)
nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1',
                       'BOROUGH', 'VEHICLE TYPE CODE 1',
                       'NUMBER OF PERSONS INJURED']]
nyc_small.head(2)

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
430841,05/19/2019,Unspecified,QUEENS,Sedan,0.0
1779911,04/06/2013,Unspecified,MANHATTAN,SPORT UTILITY / STATION WAGON,0.0


In [29]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf_small.schema)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


StructType([StructField('CRASH DATE', StringType(), True), StructField('CONTRIBUTING FACTOR VEHICLE 1', StringType(), True), StructField('BOROUGH', StringType(), True), StructField('VEHICLE TYPE CODE 1', StringType(), True), StructField('NUMBER OF PERSONS INJURED', StringType(), True)])


In [30]:
#Question 1
crash_by_date = sdf_small.groupBy('CRASH DATE').count().orderBy(F.col('count').desc()).show(5)

+----------+-----+
|CRASH DATE|count|
+----------+-----+
|11/15/2018|  224|
|01/21/2014|  217|
|12/15/2017|  210|
|02/03/2014|  205|
|12/14/2018|  203|
+----------+-----+
only showing top 5 rows



In [31]:
#Question2
crashes_by_borough = sdf_small.groupBy('BOROUGH').count().orderBy(F.col('count').desc()).show(5)

+---------+------+
|  BOROUGH| count|
+---------+------+
|         |132070|
| BROOKLYN| 92591|
|   QUEENS| 78223|
|MANHATTAN| 65192|
|    BRONX| 43322|
+---------+------+
only showing top 5 rows



In [35]:
#Question3
queens_accidents = sdf_small.filter(F.col('BOROUGH') == 'QUEENS').groupBy('CONTRIBUTING FACTOR VEHICLE 1') \
                                      .count() \
                                      .orderBy(F.col('count').desc()).show(5)

+-----------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1|count|
+-----------------------------+-----+
|                  Unspecified|27318|
|         Driver Inattentio...|16548|
|         Failure to Yield ...| 6763|
|             Backing Unsafely| 3878|
|         Following Too Clo...| 2922|
+-----------------------------+-----+
only showing top 5 rows



In [38]:
#Question4
avg_injuries = sdf_small.groupBy('BOROUGH', 'VEHICLE TYPE CODE 1') \
                        .agg(F.avg(F.col('NUMBER OF PERSONS INJURED')).alias('avg_injuries')) \
                        .orderBy(F.col('avg_injuries').desc()).show(5)



+---------+-------------------+------------+
|  BOROUGH|VEHICLE TYPE CODE 1|avg_injuries|
+---------+-------------------+------------+
|MANHATTAN|               Limo|        13.0|
| BROOKLYN|              FRONT|        12.0|
|    BRONX|              PASSE|        11.0|
|   QUEENS|                rmb|        11.0|
|MANHATTAN|              MTA B|         9.0|
+---------+-------------------+------------+
only showing top 5 rows

